In [2]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import type_score,get_year_ipcc,get_year_ipbes,get_countries,check_doi_glutton,normalize
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache

In [3]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')

In [4]:
df_ipcc=df_ipcc.dropna(subset=['doi'])

In [5]:
pd.Series([list(pd.Series([x[i].get('wg') for i in range(len(x))]).drop_duplicates()) for x in list(df_ipcc.drop_duplicates(subset=['doi']).dropna(subset=['doi']).wg_chap)]).value_counts()

[2]                   21803
[3]                   12324
[1]                   10478
[2_cross]              3491
[2, 2_cross]           1531
[1, 2]                 1459
[2, 3]                 1226
[1, 2, 2_cross]         251
[1, 3]                  189
[1, 2_cross]            176
[1, 2, 3]               172
[2, 2_cross, 3]          94
[2_cross, 3]             34
[1, 2, 2_cross, 3]       24
[1, 2_cross, 3]           6
Name: count, dtype: int64

In [6]:
min([int(x) for x in list(df_ipcc.year_OA.drop_duplicates().dropna())])

1686

In [6]:
df_ipcc.test_glutton.value_counts()

test_glutton
True    74098
Name: count, dtype: int64

In [7]:
df_ipcc=df_ipcc.rename(columns={'wg_chap':'ipcc'})

In [9]:
df_ipcc.columns

Index(['year', 'url', 'doi', 'title', 'author', 'freq', 'wg', 'chap',
       'countries', 'concepts', 'sdg', 'topics', 'is_OA_available', 'title_OA',
       'year_OA', 'authors_name', 'rors', 'institutions_names',
       'locations_names', 'ipcc', 'test_glutton'],
      dtype='object')

In [12]:
df_ipcc_group=df_ipcc.groupby(by=['doi','year'], dropna=True).agg({'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0], 'ipcc': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'freq':'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [14]:
df_ipcc_group.rors[0]

[[{'https://ror.org/05cf8a891': 'US'}],
 [{'https://ror.org/00y4zzh67': 'US'}, {'https://ror.org/04mrb6c22': 'US'}],
 []]

In [15]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [18]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [19]:
df_ipbes.columns

Index(['doi', 'title', 'date', 'creators', 'tags', 'countries', 'concepts',
       'sdg', 'topics', 'is_OA_available', 'year_OA', 'authors_name', 'rors',
       'institutions_names', 'locations_names', 'year'],
      dtype='object')

In [20]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: [{'chapter': str(x['tag']).replace('ipbes-global_chapter','')} for x in row['tags']], axis=1)

In [25]:
df_ipbes_group=df_ipbes.groupby(by=['doi','year'], dropna=True).agg({'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0],'ipbes': lambda x: list(x)[0], 'countries': lambda x: list(x)})
df_ipbes_group=df_ipbes_group.reset_index()

In [26]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [27]:
df_ipbes_group

,doi,year,rors,authors_name,institutions_names,ipbes,countries
0,10.1001/archpedi.161.7.718,2008,[[]],[{'Tina Kelley': []}],[[]],[{'chapter': '5'}],None
1,10.1002/(sici)1099-0771(199612)9:4<231::aid-bd...,1996,"[[{'https://ror.org/047426m28': 'US'}], [{'htt...","[{'Kristine M. Kuhn': ['US']}, {'Janet A. Snie...",[[{'University of Illinois Urbana-Champaign': ...,[{'chapter': '4'}],[US]
2,10.1002/16-0188.1,2016,"[[{'https://ror.org/05hs6h993': 'US'}], [], [{...","[{'Jianguo Liu': ['US']}, {'Wu Yang': []}, {'S...","[[{'Michigan State University': 'US'}], [], [{...","[{'chapter': '2.1_drivers'}, {'chapter': '5'}]",[US]
3,10.1002/2013ef000188,2014,"[[{'https://ror.org/030syve83': 'FR'}, {'https...","[{'Anny Cazenave': ['FR']}, {'Gonéri Le Cozann...","[[{'Observatoire Midi-Pyrénées': 'FR'}, {'Labo...",[{'chapter': '2.1_drivers'}],[FR]
4,10.1002/2013gb004654,2013,"[[{'https://ror.org/03hkr1v69': 'DE'}], [{'htt...","[{'Florian Schierhorn': ['DE']}, {'Daniel Müll...",[[{'Leibniz Institute of Agricultural Developm...,[{'chapter': '2.2_nature'}],[DE]
...,...,...,...,...,...,...,...
6105,10.7770/cuhso-v27n1-art1134,2017,[[{'https://ror.org/04h4z8k05': 'FR'}]],[{'Dorian Rommens': ['FR']}],"[[{'United Nations Educational, Scientific and...",[{'chapter': '3'}],[FR]
6106,10.7930/j0x63jt0,2014,"[[], [], [], [], [], [], [], [], []]","[{'J. N. Galloway': []}, {'William H. Schlesin...","[[], [], [], [], [], [], [], [], []]",[{'chapter': 'ipbes-global_glossary'}],None
6107,10.9755/ejfa.v25i1.15376,2013,[[{'https://ror.org/02d2m2044': 'US'}]],[{'Abdullah A. Jaradat': ['US']}],[[{'Agricultural Research Service': 'US'}]],[{'chapter': '3'}],[US]
6108,10.9758/cpn.2015.13.3.239,2015,"[[{'https://ror.org/02dzjmc73': 'TR'}], [{'htt...","[{'Alper Evrensel': ['TR']}, {'Mehmet Emin Cey...","[[{'Üsküdar University': 'TR'}], [{'Üsküdar Un...",[{'chapter': '2.3_ncp'}],[TR]


In [40]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')
df_ipcc_ipbes=df_ipcc_ipbes[['doi', 'year_x', 'rors_x', 'authors_name_x', 'institutions_names_x',
       'ipcc', 'countries_x', 'ipbes']].rename(columns={'year_x':'year', 'rors_x':'rors','authors_name_x':'authors_name','institutions_names_x':'institutions_names', 'countries_x':'countries'})

In [29]:
update_bso_publications()

2024-03-26 15:24:52,990 | code_utils.BSO | DEBUG | update bso publications files


DEBUG:code_utils.BSO:update bso publications files
c:\Users\haallat\Documents\teds\code_utils\BSO.py:34: DtypeWarning: Columns (17,18,21,22,23,24,25,27,28,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bso = pd.read_csv('https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/bso_dump/bso-publications-latest.csv.gz', sep=';')


2024-03-26 15:27:54,066 | code_utils.BSO | DEBUG | writing 1813337 dois info from bso publications


DEBUG:code_utils.BSO:writing 1813337 dois info from bso publications


In [42]:
bso=pd.DataFrame(get_bso_publications())

In [43]:
df_bso=bso.transpose().reset_index()

In [44]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [45]:
df_bso

,doi,rors,bso_local_affiliations
0,10.47421/sllmoo23_51-66,[04zmssz18],[130008121]
1,10.26434/chemrxiv.14095699.v2,[04zmssz18],[130008121]
2,10.26434/chemrxiv.13713328.v1,[04zmssz18],[130008121]
3,10.1101/2021.03.01.433388,[04zmssz18],[130008121]
4,10.20944/preprints202107.0318.v1,[04zmssz18],[130008121]
...,...,...,...
1813332,10.1038/srep30379,"[0495fxg12, 04w6kn183]","[775684897, 784314064, gip-be, sante_pub_inser..."
1813333,10.1038/s41567-020-0989-3,"[055ss7a31, 013cjyk83, 02en5vm52]","[199411684F, 130026149, 303623, 130023385, pcc..."
1813334,10.1101/147884,[],[]
1813335,10.1162/qss_a_00146,[],[]


In [51]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left').rename(columns={'rors_x':'rors'})

In [52]:
del df_ipcc_ipbes_bso['rors_y']

In [53]:
df_ipcc_ipbes_bso.columns

Index(['doi', 'year', 'rors', 'authors_name', 'institutions_names', 'ipcc',
       'countries', 'ipbes', 'bso_local_affiliations'],
      dtype='object')

In [54]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso_countries.json')

FileNotFoundError: File c:\Users\haallat\Documents\teds\IPCC_bibliography\AR6\structured_data\iso_countries.json does not exist

In [ ]:
df_iso=df_iso.rename(columns={'code':'countries'})

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_iso, on='countries', how='left')
del df_ipcc_ipbes_bso['countries']
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso.rename(columns={'name':'countries'})

In [ ]:
df_brevet_doi=pd.DataFrame('brevet_doi.json')

In [ ]:
df_brevet_doi

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_brevet_doi, on='doi', how='left')

In [1]:
df_ipcc_ipbes_bso

NameError: name 'df_ipcc_ipbes_bso' is not defined

In [55]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

54821 data in cached openalex data
6607 data in cached openalex data


In [56]:
df_cache

,doi,OpenAlex_data
0,10.1098/rstl.1686.0026,"[{'id': 'https://openalex.org/W4244279015', 'd..."
1,10.1098/rstl.1735.0014,"[{'id': 'https://openalex.org/W4252425265', 'd..."
2,10.1098/rstl.1861.0001,"[{'id': 'https://openalex.org/W3091998362', 'd..."
3,10.1017/s0080456800017592,"[{'id': 'https://openalex.org/W2314134964', 'd..."
4,10.2307/2992177,"[{'id': 'https://openalex.org/W256209766', 'do..."
...,...,...
60244,10.9758/cpn.2015.13.3.239,"[{'id': 'https://openalex.org/W2221413160', 'd..."
60245,10.9774/gleaf.978-1-907643-06-4_5,"[{'id': 'https://openalex.org/W3124428460', 'd..."
60246,10.9783/9780812295948-002,"[{'id': 'https://openalex.org/W3137011924', 'd..."
60247,10.2172/2283221,"[{'id': 'https://openalex.org/W4391458189', 'd..."


In [57]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [58]:
dict_doi_bso=[]

In [59]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [60]:
len(dict_doi_bso)

58409

In [61]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [62]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes2.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [ ]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes2.jsonl', lines=True)

In [ ]:
dd2=dd.dropna(subset=['ipcc']).drop_duplicates(subset=['doi'])

In [ ]:
len(dd2.loc[(dd2['ipcc'].apply(lambda wg_chap: any('1' in x.get('wg', '') for x in wg_chap)))])

In [ ]:
current_dict = dd.to_dict(orient='records')

In [ ]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [ ]:
df_elastic_input=pd.DataFrame(current_dict)

In [ ]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [ ]:
del df_elastic_input['openalex_abstract_inverted_index']

In [ ]:
len(df_elastic_input)

In [ ]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

In [ ]:
#df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [ ]:
es_url_without_http = os.getenv("ES_URL").replace('https://','').replace('http://','')
es_host = f'https://{os.getenv("ES_LOGIN_TEDS_BACK")}:{parse.quote(os.getenv("ES_PASSWORD_TEDS_BACK"))}@{es_url_without_http}'
index_name='teds-bibliography'